In [1]:
import pandas as pd
import time
from scipy import stats
from tqdm import tqdm
import numpy as np
from pathlib import Path
import snf
from sklearn.cluster import spectral_clustering
from sklearn.metrics import v_measure_score
import networkx as nx

### Different networks data processing 

In [2]:
def get_correlation_dataframe(df):
    col_1 = []
    col_2 = []
    col_r = []
    col_p = []
    
    for idx1, row1 in tqdm(df.iterrows(), total=df.shape[0]):
        for idx2, row2 in df.loc[idx1:, :].iterrows():
            r, p = stats.pearsonr(row1.values, row2.values)
            col_1.append(idx1)
            col_2.append(idx2)
            col_r.append(r)
            col_p.append(p)
            
    corr_df = pd.DataFrame.from_dict({
        "sample1": col_1,
        "sample2": col_2,
        "r": col_r,
        "p": col_p
    })
    return corr_df


def merge_correlation_dataframes(dfs):
    
    greatest_r = np.argmax(np.array([df.r for df in dfs]), axis=0)
    to_concat = [df.loc[greatest_r == i] for i, df in enumerate(dfs)]
    return pd.concat(to_concat).sort_index()


def build_edge_list(df, r_filter, p_filter):
    edges_df = df.loc[(df.r >= r_filter) & (df['p'] <= p_filter)]
    return edges_df.rename(columns={'sample1':'source', 'sample2':'target'})

def filter_relevant_connections(df, threshold):
    return df.loc[(df.weight >= threshold)]


def get_stage_class_from_patient(patient_idx, clin_df, agglutinate_stages=False):
    stage_str = clin_df.loc[patient_idx, "pathologic_stage"]
    
    if stage_str in ["stage i"+suffix for suffix in ['', 'a','b','c']]:
        return "stage1"
    elif stage_str in ["stage ii"+suffix for suffix in ['', 'a','b','c']]:
        if agglutinate_stages:
            return "stage23"
        else:
            return "stage2"
    elif stage_str in ["stage iii"+suffix for suffix in ['', 'a','b','c']]:
        if agglutinate_stages:
            return "stage23"
        else:
            return "stage3"
    elif stage_str in ["stage iv"+suffix for suffix in ['', 'a','b','c']]:
        return "stage4"
    else:
        return np.nan


def build_class_df(sample_idxs, agglutinate_stages=False):
    
    clin_df = pd.read_csv(f"{base}{cancer}_clin.txt", sep="\t", index_col=0).T.iloc[:, [6]]
    
    class_col = []
    for idx in sample_idxs:
        patient_idx = '-'.join(idx.split('.')[:-1]).lower()
        sample_type = int(idx.split('.')[-1])
        
        if sample_type <= 9:   # Tumor sample
            class_col.append(get_stage_class_from_patient(patient_idx, clin_df, agglutinate_stages))
        elif sample_type <= 19:   # Normal sample
            class_col.append('normal')
        elif sample_type <= 29:   # Control sample
            print(f"Warning! Found control sample {idx}, Skipping...")
            continue
        else:
            print(f"Warning! Found unexpected sample type: {idx}. Skipping...")
                  
    return pd.DataFrame.from_dict({
            "id": sample_idxs,
            "class": class_col
        }).set_index("id")


def get_consistency_index(corr_df, class_df):
    correct_connections = 0
    for index, row in corr_df.iterrows():
        src_class = class_df.loc[row[0], "class"]
        trg_class = class_df.loc[row[1], "class"]
        
        if src_class == trg_class:
            correct_connections += 1
    return correct_connections/len(corr_df)


def generate_csvs(edges_df, class_df, max_each_feature=100, multi_omics=True):
    
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    
    edges_df.to_csv(save_dir+"edges.csv", index=False)
    class_df.to_csv(save_dir+"classes.csv", index=True)
    
    gene = pd.read_csv(f"{base}{cancer}_mRNA.csv", index_col=0).iloc[:max_each_feature, :]
    if multi_omics:
        mirna = pd.read_csv(f"{base}{cancer}_miRNA.csv", index_col=0).iloc[:max_each_feature, :]
        meth = pd.read_csv(f"{base}{cancer}_Methy.csv", index_col=0).iloc[:max_each_feature, :] 
        #cnv = pd.read_csv(f"{base}{cancer}_CNV.csv", index_col=0).iloc[:max_each_feature, :]
        #features_df = pd.concat([gene,mirna,meth,cnv]).T
        features_df = pd.concat([gene,mirna,meth]).T
    else:
        features_df = gene.T
    
    features_df.loc[class_df.index, :].to_csv(save_dir+"features.csv", index=True)
    return


def get_snf_network(dfs_values_matrices, class_df):
    affinity_networks = snf.make_affinity(dfs_values_matrices)
    fused_network = snf.snf(affinity_networks)
    np.fill_diagonal(fused_network, 1)
    G = nx.from_pandas_adjacency(pd.DataFrame(data=fused_network, index=class_df.index.values, columns=class_df.index.values), create_using=nx.Graph())
    return nx.to_pandas_edgelist(G)

In [3]:
cancers = ["COAD", "KIRC", "LUAD"]
percentiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

for cancer in cancers:
    print(cancer)
    base = f"C:/Users/colombelli/Desktop/TCC/experiments/{cancer}/"

    dfs_values = []

    df1 = pd.read_csv(f"{base}{cancer}_mRNA.csv", index_col=0).T
    class_df = build_class_df(list(df1.index), agglutinate_stages=False).dropna()
    dfs_values.append(df1.loc[class_df.index, :].values)

    df2 = pd.read_csv(f"{base}{cancer}_miRNA.csv", index_col=0).T
    dfs_values.append(df2.loc[class_df.index, :].values)

    df3 = pd.read_csv(f"{base}{cancer}_Methy.csv", index_col=0).T
    dfs_values.append(df3.loc[class_df.index, :].values)
    
    graph_df = get_snf_network(dfs_values, class_df)
    
    # Write class counts and their weights for the categorical cross entropy
    n_samples = len(class_df)
    n_classes = 4
    with open(f"{base}class_info.txt", 'w') as f:
        print("class\t | #\t| weight", file=f)
        for i, count in class_df.value_counts().items():
            weight = n_samples / (n_classes * count)
            print(i[0], "\t |", count, "\t| ", weight, file=f)
    
    
    for percentile in percentiles:
        print("Percentile: ", percentile)
        th = graph_df['weight'].quantile(percentile)
        save_dir = f"{base}snf/{str(percentile).replace('.', '')}/"
        generate_csvs(filter_relevant_connections(graph_df, th), class_df, max_each_feature=999999)
    print("\n\n")

COAD
Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99



KIRC
Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99



LUAD
Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99





In [4]:
cancers = ["COAD", "KIRC", "LUAD"]
percentiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

for cancer in cancers:
    print(cancer)
    base = f"C:/Users/colombelli/Desktop/TCC/experiments/{cancer}/"

    df = pd.read_csv(f"{base}{cancer}_mRNA.csv", index_col=0).T
    class_df = build_class_df(list(df.index), agglutinate_stages=False).dropna()
    df = df.loc[class_df.index, :]
    corr_df_mrna = get_correlation_dataframe(df) 

    df = pd.read_csv(f"{base}{cancer}_miRNA.csv", index_col=0).T
    df = df.loc[class_df.index, :]
    corr_df_mirna = get_correlation_dataframe(df) 

    df = pd.read_csv(f"{base}{cancer}_Methy.csv", index_col=0).T
    df = df.loc[class_df.index, :]
    corr_df_methy = get_correlation_dataframe(df) 

    corr_df = merge_correlation_dataframes([corr_df_mrna, corr_df_mirna, corr_df_methy])
    
    # Write class counts and their weights for the categorical cross entropy
    n_samples = len(class_df)
    n_classes = 4
    with open(f"{base}class_info.txt", 'w') as f:
        print("class\t | #\t| weight", file=f)
        for i, count in class_df.value_counts().items():
            weight = n_samples / (n_classes * count)
            print(i[0], "\t |", count, "\t| ", weight, file=f)
    
    
    for percentile in percentiles:
        print("Percentile: ", percentile)
        th = corr_df['r'].quantile(percentile)
        save_dir = f"{base}correlation_multi_omics/{str(percentile).replace('.', '')}/"
        generate_csvs(build_edge_list(corr_df, th, 0.05), class_df, max_each_feature=999999)
    print("\n\n")

COAD


100%|████████████████████████████████████████████████████████████████████████████████| 282/282 [00:03<00:00, 86.06it/s]


Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99



KIRC


100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:04<00:00, 76.71it/s]


Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99



LUAD


100%|████████████████████████████████████████████████████████████████████████████████| 445/445 [00:08<00:00, 52.04it/s]


Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99





In [5]:
cancers = ["COAD", "KIRC", "LUAD"]
percentiles = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

for cancer in cancers:
    print(cancer)
    base = f"C:/Users/colombelli/Desktop/TCC/experiments/{cancer}/"

    df = pd.read_csv(f"{base}{cancer}_mRNA.csv", index_col=0).T
    class_df = build_class_df(list(df.index), agglutinate_stages=False).dropna()
    df = df.loc[class_df.index, :]
    corr_df = get_correlation_dataframe(df)
    
    # Write class counts and their weights for the categorical cross entropy
    n_samples = len(class_df)
    n_classes = 4
    with open(f"{base}class_info.txt", 'w') as f:
        print("class\t | #\t| weight", file=f)
        for i, count in class_df.value_counts().items():
            weight = n_samples / (n_classes * count)
            print(i[0], "\t |", count, "\t| ", weight, file=f)
    
    
    for percentile in percentiles:
        print("Percentile: ", percentile)
        th = corr_df['r'].quantile(percentile)
        save_dir = f"{base}correlation/{str(percentile).replace('.', '')}/"
        generate_csvs(build_edge_list(corr_df, th, 0.05), class_df, max_each_feature=999999)
    print("\n\n")

COAD


100%|████████████████████████████████████████████████████████████████████████████████| 282/282 [00:03<00:00, 82.73it/s]


Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99



KIRC


100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:04<00:00, 74.84it/s]


Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99



LUAD


100%|████████████████████████████████████████████████████████████████████████████████| 445/445 [00:08<00:00, 52.73it/s]


Percentile:  0.01
Percentile:  0.05
Percentile:  0.1
Percentile:  0.25
Percentile:  0.5
Percentile:  0.75
Percentile:  0.9
Percentile:  0.95
Percentile:  0.99



